---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [82]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
#print(len(df))
print(df.iloc[77])
#print(df.iloc[73])

01/05/1999 [report_end]



In [89]:
def date_sorter():
    import re
    mon2int = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 
               'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    re1 = r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})'
    
    #Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    re2 = r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*)[.]?[ -](?P<day>\d{1,2})(?:\w{2})?[,]?[ -](?P<year>\d{4})'
    
    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    re3 = r'(?P<day>\d{1,2}) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*)[.,]? (?P<year>\d{4})'
    
    # Feb 2009; Sep 2009; Oct 2010
    re4 = r'(?P<day>)(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*)[,]?[ -](?P<year>\d{4})'
    
    # 6/2008; 12/2009
    re5 = r'(?P<month>\d{1,2})(?P<day>)[/-](?P<year>\d{4})'
    
    #2009; 2010
    re6 = r'(?P<month>)(?P<day>)(?P<year>\d{4})'
    
    df1=df.str.extractall(re1)
    df1.index = df1.index.droplevel(1)
    df1['year']=df1['year'].apply(lambda x: '19'+x if len(x)==2 else x)
    df1=df1.astype('int64')
    df1=df1[(df1['day']<32) & (df1['month']<13)]

    df2=df.str.extractall(re2)
    df2.index = df2.index.droplevel(1)
    df2['month']=df2['month'].apply(lambda x: mon2int[x[:3]])
    df2=df2.astype('int64')
    df2=df2[(df2['day']<32) & (df2['month']<13)]
    
    df3=df.str.extractall(re3)
    df3.index = df3.index.droplevel(1)
    df3['month']=df3['month'].apply(lambda x: mon2int[x[:3]])
    df3=df3.astype('int64')
    df3=df3[(df3['day']<32) & (df3['month']<13)]
    
    df_re=df1.append(df2)
    df_re=df_re.append(df3)
    
    df4=df.str.extractall(re4)
    df4.index = df4.index.droplevel(1)
    df4['day']=df4['day'].fillna(1)
    df4['month']=df4['month'].apply(lambda x: mon2int[x[:3]])
    df4=df4.astype('int64')
    df4=df4[(df4['day']<32) & (df4['month']<13)]
    df_re=df_re.append(df4)
    
    df5=df.str.extractall(re5)
    df5.index = df5.index.droplevel(1)
    df5['day']=df5['day'].fillna(1)
    df5=df5.astype('int64')
    df5=df5[(df5['day']<32) & (df5['month']<13)]
    df_re=df_re.append(df5)
    
    df6=df.str.extractall(re6)
    df6.index = df6.index.droplevel(1)
    df6['day']=df6['day'].fillna(1)
    df6['month']=df6['month'].fillna(1)
    df6=df6.astype('int64')
    df_re=df_re.append(df6)
    
    output = df1.append(df2[~df2.index.isin(df1.index)])
    output = output.append(df3[~df3.index.isin(output.index)])
    output = output.append(df4[~df4.index.isin(output.index)])
    output = output.append(df5[~df5.index.isin(output.index)])
    output = output.append(df6[~df6.index.isin(output.index)])
    
    output_index = output.sort_values(['year','month','day'], axis=0).reset_index()
    
    return output_index['index']

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64

In [ ]:
# practice version
def date_sorter():
    import re
    mon2int = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 
               'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    re1 = r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})'
    
    #Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    re2 = r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*)[.]?[ -](?P<day>\d{1,2})(?:\w{2})?[,]?[ -](?P<year>\d{4})'
    
    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    re3 = r'(?P<day>\d{1,2}) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*)[.,]? (?P<year>\d{4})'
    
    # Feb 2009; Sep 2009; Oct 2010
    re4 = r'(?P<day>)(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*)[,]?[ -](?P<year>\d{4})'
    
    # 6/2008; 12/2009
    re5 = r'(?P<month>\d{1,2})(?P<day>)[/-](?P<year>\d{4})'
    
    #2009; 2010
    re6 = r'(?P<month>)(?P<day>)(?P<year>\d{4})'
    
    df1=df.str.extractall(re1)
    #df1=df.str.extractall(re1)#.drop('match')#.reset_index()[['month','day','year']]
    df1.index = df1.index.droplevel(1)
    df1['year']=df1['year'].apply(lambda x: '19'+x if len(x)==2 else x)
    df1=df1.astype('int64')
    df1=df1[(df1['day']<32) & (df1['month']<13)]
    #print(df1)#.duplicated().argmax())
    #print(df1.iloc[57])
    #print(df1.iloc[58])
    
    df2=df.str.extractall(re2)
    #df2=df.str.extractall(re2)#.drop('match')#.reset_index()[['month','day','year']]
    df2.index = df2.index.droplevel(1)
    df2['month']=df2['month'].apply(lambda x: mon2int[x[:3]])
    df2=df2.astype('int64')
    df2=df2[(df2['day']<32) & (df2['month']<13)]
    #print(df2)
    
    df3=df.str.extractall(re3)
    #df3=df.str.extractall(re3)#.drop('match')#.reset_index()[['month','day','year']]
    df3.index = df3.index.droplevel(1)
    df3['month']=df3['month'].apply(lambda x: mon2int[x[:3]])
    df3=df3.astype('int64')
    df3=df3[(df3['day']<32) & (df3['month']<13)]
    #print(df3)
    
    df_re=df1.append(df2)
    df_re=df_re.append(df3)
    
    #print(len(df_re))
    df4=df.str.extractall(re4)
    #df4=df.iloc[228:].str.extractall(re4)
    df4.index = df4.index.droplevel(1)
    df4['day']=df4['day'].fillna(1)
    df4['month']=df4['month'].apply(lambda x: mon2int[x[:3]])
    df4=df4.astype('int64')
    df4=df4[(df4['day']<32) & (df4['month']<13)]
    df_re=df_re.append(df4)
    #print(df4)
    #print(df_re)
    
    df5=df.str.extractall(re5)
    #df5=df[343:].str.extractall(re5)#.drop('match')#.reset_index()[['month','day','year']]
    df5.index = df5.index.droplevel(1)
    df5['day']=df5['day'].fillna(1)
    df5=df5.astype('int64')
    df5=df5[(df5['day']<32) & (df5['month']<13)]
    df_re=df_re.append(df5)
    
    df6=df.str.extractall(re6)
    #df6=df[455:].str.extractall(re6)#.drop('match')#.reset_index()[['month','day','year']]
    df6.index = df6.index.droplevel(1)
    #df5['year']=df5['year'].apply(lambda x: '19'+x if len(x)==2 else x)
    df6['day']=df6['day'].fillna(1)
    df6['month']=df6['month'].fillna(1)
    df6=df6.astype('int64')
    df_re=df_re.append(df6)
    #print(df_re)
    
    index_df = df_re.sort_values(['year','month','day'], axis=0).reset_index()
    #print(index_df)
    
    output = df1.append(df2[~df2.index.isin(df1.index)])
    output = output.append(df3[~df3.index.isin(output.index)])
    output = output.append(df4[~df4.index.isin(output.index)])
    output = output.append(df5[~df5.index.isin(output.index)])
    output = output.append(df6[~df6.index.isin(output.index)])
    
    output_index = output.sort_values(['year','month','day'], axis=0).reset_index()
    print(output_index['index'])
    
    #return index_df['index']

date_sorter()